In [58]:
import os
from pinecone import Pinecone

from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [5]:
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

In [7]:
pc.list_indexes()

{'indexes': []}

In [11]:
index_name = "namespace-test-index"

# インデックスはコンソールから作成できるのでそのほうがいい
# pc.create_index(
#     name=index_name,
#     dimension=1536,
#     metric="cosine",
#     spec=ServerlessSpec(cloud="aws", region="us-east-1")
# )

In [22]:
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'namespace-test-index-k6ehddk.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'namespace-test-index',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [23]:
index = pc.Index(index_name)
index

In [43]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

In [18]:
contents = [
    "新入社員は初日に人事部で手続きを行ってください。",
    "製品Aの電源ボタンは右側面にあります。",
    "パスワードの変更は3ヶ月ごとに必要です。",
    "太陽光パネルの効率は15%から20%です。",
    "契約書の有効期限は署名日から2年間です。",
    "新薬Xは高血圧治療に効果があります。",
    "第3四半期の売上は前年比10%増加しました。",
    "患者Yさんはペニシリンにアレルギーがあります。",
    "台風5号は明日午後に本州に上陸する見込みです。",
    "商品の返品は購入後14日以内に限ります。"
]

In [25]:
from langchain_core.documents import Document

documents = []
for content in contents:
    documents.append(Document(page_content=content))

# Pineconeにドキュメントを追加
vectorstore.add_documents(documents, namespace="test1")

['f388d5c2-63c6-43af-a095-dbb2324fe89c',
 '0b687e9b-ff2e-4c5b-bcb2-95e9490e621b',
 '5e4efb87-4c2f-4efe-9c99-26936d9fb5d5',
 '587af52f-8a69-4496-a57e-f046323b285c',
 '59b25b88-2957-4fdb-8cca-8ca75df8babf',
 'd3051c53-ea51-47e8-9111-b31f83e80644',
 'd29bc8eb-bbd2-458a-a60f-23c63125794d',
 'd8b93c87-b502-48f6-a500-87b04c19395f',
 '906c1919-a0d2-40fe-95b5-89c825c472a8',
 '67c9b15a-2e8d-41d7-94c9-d34cb92b6637']

In [39]:
contents2 = [
    "年次有給休暇は勤続6ヶ月後から付与されます。",
    "ノートPCのバッテリー持続時間は約8時間です。",
    "社内LANのパスワードは毎月1日に更新されます。",
    "電気自動車の航続距離は満充電で約400kmです。",
    "特許権の存続期間は出願から20年間となります。",
    "新型コロナワクチンは2回接種が基本となります。",
    "当社株価は前日終値比3%高の1,200円で取引開始。",
    "患者Aさんの血圧は収縮期140、拡張期90です。",
    "来月の展示会は東京ビッグサイトで3日間開催。",
    "製品保証期間は購入日から1年間有効です。"
]

documents = []
for content in contents2:
    documents.append(Document(page_content=content))

vectorstore.add_documents(documents, namespace="test2")

['f1bf0a80-ad0e-4052-8916-11b9b5267e0f',
 'f4c1ebc9-755d-4bda-b8e2-cd06a3052877',
 '066f965e-87b5-491a-a195-63862adde473',
 'e517d572-1df8-4b19-a2e3-7d9f73a5709d',
 'f1d646a6-3274-4037-ac25-27886001e204',
 '55daaaee-24ba-4b15-9fef-1d7c27366fea',
 '081e0425-be9a-4640-8aeb-ecd2df5f9ca5',
 'c056c33a-d631-451f-aa04-e9c81bfbed09',
 '83a3b413-fc3e-4cf3-a1c8-4dd28628ef1f',
 '5851946d-be0c-488d-a05d-543b67438788']

In [44]:
results = vectorstore.similarity_search(
    "年次休暇の人事規定について教えて",
    k=5,
    namespace="test1"
)

results

[Document(page_content='新入社員は初日に人事部で手続きを行ってください。'),
 Document(page_content='契約書の有効期限は署名日から2年間です。'),
 Document(page_content='パスワードの変更は3ヶ月ごとに必要です。'),
 Document(page_content='第3四半期の売上は前年比10%増加しました。'),
 Document(page_content='商品の返品は購入後14日以内に限ります。')]

In [45]:
results = vectorstore.similarity_search(
    "年次休暇の人事規定について教えて",
    k=5,
    namespace="test2"
)

results

[Document(page_content='年次有給休暇は勤続6ヶ月後から付与されます。'),
 Document(page_content='特許権の存続期間は出願から20年間となります。'),
 Document(page_content='社内LANのパスワードは毎月1日に更新されます。'),
 Document(page_content='製品保証期間は購入日から1年間有効です。'),
 Document(page_content='新型コロナワクチンは2回接種が基本となります。')]

In [51]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2, "namespace": "test1"})
retriever.invoke("年次休暇の人事規定について教えて")

[Document(page_content='新入社員は初日に人事部で手続きを行ってください。'),
 Document(page_content='契約書の有効期限は署名日から2年間です。')]

In [53]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2, "namespace": "test2"})
retriever.invoke("年次休暇の人事規定について教えて")

[Document(page_content='年次有給休暇は勤続6ヶ月後から付与されます。'),
 Document(page_content='特許権の存続期間は出願から20年間となります。')]

In [63]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2, "namespace": "test1"})
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model="gpt-4o-mini")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("パスワードの変更は何か月ごとに必要ですか？")

'パスワードの変更は3ヶ月ごとに必要です。'